In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
import pickle
import pandas as pd
import glob, os, gc
from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.model_selection import KFold

# Add the path to the directory containing the module
import sys
sys.path.append('../../')
from util.ml import baseline, metrics, nestedMLR

from properscoring import crps_ensemble  # For CRPS calculation
from sklearn.utils import resample  # For bootstrapping

In [2]:
# Find the folder name organized by seed number
seed_doc = sorted(glob.glob('../../datas/seed_revised_*/'))[0]

# Load the data
# Load the time series data
df = pd.read_csv(seed_doc +'X_train_ts_all.csv')
df_valid = pd.read_csv(seed_doc +'X_validation_ts_all.csv')
df_test = pd.read_csv(seed_doc +'X_test_ts_all.csv')
# Find the name for each column
column_names = ([obj.split('_step_')[0] for obj in df.columns])
# Unique names in the column name list
unique_names = list(set(column_names))
unique_names_filt = [obj for obj in unique_names if 'large_scale' not in obj]

pcs_train = baseline.load_pickle(f'../../datas/proc/sfs/pcsall_train.pkl')
pcs_val = baseline.load_pickle(f'../../datas/proc/sfs/pcsall_valid.pkl')

# Now we read in the y data for every fold
y_train = []
y_val = []
for i in range(7):
    y_train.append(baseline.load_pickle(f'../../datas/proc/sfs/y/ytrain_split_{i}.pkl'))
    y_val.append(baseline.load_pickle(f'../../datas/proc/sfs/y/yval_split_{i}.pkl'))

# Load the test data
y_test = baseline.load_pickle('../../datas/proc/sfs/y/ytest.pkl')
y_testz = [y_test for i in range(7)]

In [3]:
from scipy.optimize import minimize
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [4]:
# List of variable names
var_names = list(pcs_train[0].keys())

# Empty list to store the chosen variables
selected_vars = []
# List to store all the variable choices
remaining_vars = [var for var in var_names if "large_scale" not in var].copy()
# Initial RMSE to beat is infinity
best_val_rmse = float('inf')
# Target transformation category
target_cat = 'max'
# Seed
seed = 42
# Hyperparameter space for the random forest
param_grid = {
    "n_estimators": [100],
    "max_depth": [None],
    "min_samples_split": [2],
}

In [5]:
from natsort import natsorted
def r2_score_f(y_true,y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    r2 = 1-np.sum((y_true-y_pred)**2)/np.sum((y_true-np.mean(y_true))**2)
    return r2

## Linear

### all

In [11]:
# Initialize variables
selected_pcs = []  # List to store selected PCs
remaining_vars = [var for var in var_names if "large_scale" not in var].copy()  # All variables initially available for selection
tofilt = ['_10_', '_20_', '_30_', '_50_', '_70_']
remaining_vars = [var for var in remaining_vars if not any(substring in var for substring in tofilt)] # Filter out specific pressure levels
best_val_rmse = float('inf')  # Start with a very high RMSE value

# Track RMSEs
rmse_log = []  # List to log RMSE values for each iteration

while remaining_vars:
    best_var = None
    best_pc_index = None
    #best_iseed = None
    best_mean_val_rmse = best_val_rmse
    best_mean_train_rmse = None

    # Test each variable
    for var in remaining_vars:
        nPC = pcs_train[0][var].shape[1]  # Number of PCs for this variable

        # Test each PC of the variable
        for pc_index in range(nPC):
            train_scores = []
            mean_score = []

            # Evaluate using all seeds
            for iseed in range(7):
                # Prepare data for the current candidate PC
                candidate_features = [pcs_train[iseed][sel_var][:, [pc_idx]] 
                                      for sel_var, pc_idx in selected_pcs]
                candidate_features.append(pcs_train[iseed][var][:, [pc_index]])
                X_train_subset = np.hstack(candidate_features)
                X_val_subset = np.hstack(
                    [pcs_val[iseed][sel_var][:, [pc_idx]] for sel_var, pc_idx in selected_pcs] +
                    [pcs_val[iseed][var][:, [pc_index]]]
                )

                # Train a model and evaluate
                model = linear_model.LinearRegression()
                model.fit(X_train_subset, y_train[iseed][target_cat])
                y_pred = model.predict(X_val_subset)

                # Training RMSE
                y_train_pred = model.predict(X_train_subset)
                train_rmse = mean_squared_error(y_train[iseed][target_cat], y_train_pred, squared=False)
                #train_rmse = -r2_score_f(y_train[iseed][target_cat], y_train_pred)
                train_scores.append(train_rmse)

                # Calculate the validation RMSE
                val_rmse = mean_squared_error(y_val[iseed][target_cat], y_pred, squared=False)
                #val_rmse = -r2_score_f(y_val[iseed][target_cat], y_pred)
                mean_score.append(val_rmse)

            # Compute the mean training RMSE across seeds
            mean_train_rmse = np.mean(train_scores)
            # Compute the mean validation RMSE across seeds
            mean_val_rmse = np.mean(mean_score)

            # Update the best PC if this one performs better
            if mean_val_rmse < best_mean_val_rmse:
                best_mean_val_rmse = mean_val_rmse
                best_mean_train_rmse = mean_train_rmse
                best_var = var
                best_pc_index = pc_index
                #best_iseed = current_best_iseed

    # Check if we found a PC that improves validation RMSE
    if best_var and best_mean_val_rmse < best_val_rmse:
        # Add the best-performing PC to the selected set
        selected_pcs.append((best_var, best_pc_index))
        remaining_vars.remove(best_var)
        best_val_rmse = best_mean_val_rmse

        # Log RMSEs for this iteration
        rmse_log.append({
            "selected_pc": f"{best_var}_PC{best_pc_index + 1}",
            "train_rmse": best_mean_train_rmse,
            "val_rmse": best_mean_val_rmse,
        })
        print(f"Selected PC: {best_var}_PC{best_pc_index + 1}, Train RMSE: {best_mean_train_rmse}, Val RMSE: {best_mean_val_rmse}")
    else:
        print("No improvement. Stopping feature selection.")
        break

# Train the final model using all selected PCs and all training data
final_X_train = np.hstack(
    [pcs_train[0][sel_var][:, [pc_idx]] for sel_var, pc_idx in selected_pcs]
)
final_model = linear_model.LinearRegression()
final_model.fit(final_X_train, y_train[0][target_cat])

# Save the final model
#baseline.save_models(final_model,f'../../datas/proc/sfs/results/best_linear_max_model_r2.pkl')
#baseline.save_models(selected_pcs,f'../../datas/proc/sfs/results/best_linear_max_feature_r2.pkl')
#baseline.save_models(rmse_log,f'../../datas/proc/sfs/results/best_linear_max_R2log.pkl')

Selected PC: geopotential_500_min_PC1, Train RMSE: 5.49161205270351, Val RMSE: 5.469222314457427
Selected PC: 100m_magnitude_of_wind_min_PC4, Train RMSE: 5.2722813353341085, Val RMSE: 5.358255958740794
Selected PC: mean_top_net_long_wave_radiation_flux_min_PC1, Train RMSE: 5.04590101911309, Val RMSE: 5.268538471732891
Selected PC: surface_sensible_heat_flux_max_PC1, Train RMSE: 4.877946861130859, Val RMSE: 5.172161830022704
Selected PC: mean_vertically_integrated_moisture_divergence_std_PC5, Train RMSE: 4.6862509642980275, Val RMSE: 5.08572205520873
Selected PC: 2m_temperature_max_PC8, Train RMSE: 4.550666728708803, Val RMSE: 4.999789528465514
Selected PC: relative_humidity_150_min_PC3, Train RMSE: 4.3616851269015235, Val RMSE: 4.923617760012242
Selected PC: relative_humidity_500_mean_PC9, Train RMSE: 4.184191076029865, Val RMSE: 4.855652274198742
Selected PC: relative_humidity_400_std_PC3, Train RMSE: 4.031929940564026, Val RMSE: 4.784958899409555
Selected PC: relative_humidity_950_mi

LinearRegression()

In [15]:
# Train the final model using all selected PCs and all training data
final_X_train = np.hstack(
    [pcs_train[0][sel_var][:, [pc_idx]] for sel_var, pc_idx in selected_pcs]
)
final_model = linear_model.LinearRegression()
final_model.fit(final_X_train, y_train[0][target_cat])

# Save the final model
baseline.save_models(final_model,f'../../datas/proc/sfs/results/best_linear_max_model.pkl')
baseline.save_models(selected_pcs,f'../../datas/proc/sfs/results/best_linear_max_feature.pkl')

### Probabilistic

In [6]:
# Initialize variables
selected_pcs = []  # List to store selected PCs
remaining_vars = list(pcs_train[0].keys())  # All variables initially available for selection
best_val_crps = float('inf')  # Start with a very high RMSE value

while remaining_vars:
    best_var = None
    best_pc_index = None
    best_iseed = None
    best_mean_val_crps = best_val_crps

    # Test each variable
    for var in remaining_vars:
        nPC = pcs_train[0][var].shape[1]  # Number of PCs for this variable

        # Test each PC of the variable
        for pc_index in range(nPC):
            mean_score = []

            # Evaluate using all seeds
            for iseed in range(7):
                # Prepare data for the current candidate PC
                candidate_features = [pcs_train[iseed][sel_var][:, [pc_idx]] 
                                      for sel_var, pc_idx in selected_pcs]
                candidate_features.append(pcs_train[iseed][var][:, [pc_index]])
                X_train_subset = np.hstack(candidate_features)
                X_val_subset = np.hstack(
                    [pcs_val[iseed][sel_var][:, [pc_idx]] for sel_var, pc_idx in selected_pcs] +
                    [pcs_val[iseed][var][:, [pc_index]]]
                )

                # Train a model and evaluate
                model = linear_model.LinearRegression()
                model.fit(X_train_subset, y_train[iseed][target_cat])
                y_pred = model.predict(X_val_subset)

                # Generate ensemble predictions using bootstrapping
                y_pred_ensemble = []
                for _ in range(100):  # Number of ensemble members
                    bootstrap_X, bootstrap_y = resample(X_train_subset, y_train[iseed][target_cat])
                    bootstrap_model = linear_model.LinearRegression()
                    bootstrap_model.fit(bootstrap_X, bootstrap_y)
                    y_pred_ensemble.append(bootstrap_model.predict(X_val_subset))
                y_pred_ensemble = np.array(y_pred_ensemble)  # Shape: (n_samples, n_ensemble)

                # Calculate the CRPS score
                val_crps = np.mean([crps_ensemble(y_val[iseed][target_cat][i], y_pred_ensemble[:, i, :].T) 
                                for i in range(len(y_val[iseed][target_cat]))])
                mean_score.append(val_crps)

                # Track the best seed for this PC
                if val_crps == min(mean_score):  # Check if this seed gives the best RMSE
                    current_best_iseed = iseed

            # Compute the mean CRPS across seeds
            mean_val_crps = np.mean(mean_score)

            # Update the best PC if this one performs better
            if mean_val_crps < best_mean_val_crps:
                best_mean_val_crps = mean_val_crps
                best_var = var
                best_pc_index = pc_index
                best_iseed = current_best_iseed

    # Check if we found a PC that improves validation RMSE
    if best_var and best_mean_val_crps < best_val_crps:
        # Add the best-performing PC to the selected set
        selected_pcs.append((best_var, best_pc_index))
        remaining_vars.remove(best_var)
        best_val_crps = best_mean_val_crps
        print(f"Selected PC: {best_var}_PC{best_pc_index + 1}, Mean Val RMSE: {best_mean_val_crps}, Seed: {best_iseed}")
    else:
        print("No improvement. Stopping feature selection.")
        break


Selected PC: mean_surface_latent_heat_flux_std_PC8, Mean Val RMSE: 4.046519506411527, Seed: 0
Selected PC: 10m_u_component_of_wind_std_PC4, Mean Val RMSE: 3.8527484877697766, Seed: 0
Selected PC: surface_latent_heat_flux_mean_PC2, Mean Val RMSE: 3.753582792237446, Seed: 0
Selected PC: mean_sea_level_pressure_max_PC9, Mean Val RMSE: 3.675957197286574, Seed: 0
Selected PC: 10m_u_component_of_wind_max_PC7, Mean Val RMSE: 3.618071848186584, Seed: 0
Selected PC: geopotential_500_mean_PC2, Mean Val RMSE: 3.5877797022992963, Seed: 0
Selected PC: mean_sea_level_pressure_mean_PC4, Mean Val RMSE: 3.5406891172095594, Seed: 0
Selected PC: surface_pressure_max_PC5, Mean Val RMSE: 3.533518653922654, Seed: 0
Selected PC: geopotential_1000_min_PC5, Mean Val RMSE: 3.525916407559397, Seed: 0
Selected PC: surface_latent_heat_flux_std_PC9, Mean Val RMSE: 3.4839081333258624, Seed: 0
Selected PC: mean_sea_level_pressure_min_PC8, Mean Val RMSE: 3.458098561269266, Seed: 0
Selected PC: geopotential_500_max_PC5

## Climatolopgy

In [10]:
meantrain_val = []
meantrain_train = []
meantrain_test = []
for i in range(7):
    meantrain_val.append([y_train[i]['max'].mean(axis=0) for _ in range(y_val[i]['cdf'].shape[0])])
    meantrain_train.append([y_train[i]['max'].mean(axis=0) for _ in range(y_train[i]['cdf'].shape[0])])
    meantrain_test.append([y_train[i]['max'].mean(axis=0) for _ in range(y_testz[i]['cdf'].shape[0])])

print(f"Test:{np.mean(np.asarray([mean_squared_error(y_testz[i]['max'], meantrain_test[i], squared=False) for i in range(7)]))}")
print(f"Validation:{np.mean(np.asarray([mean_squared_error(y_val[i]['max'], meantrain_val[i], squared=False) for i in range(7)]))}")
print(f"Training:{np.mean(np.asarray([mean_squared_error(y_train[i]['max'], meantrain_train[i], squared=False) for i in range(7)]))}")

Test:6.234292848513657
Validation:5.605705404301584
Training:5.750965193012404


In [7]:
# Parameters
n_bootstrap = 100  # Number of bootstrap iterations
n_seeds = 7        # Number of seeds (models)

# Collect CRPS scores for each bootstrap iteration
bootstrap_crps = []

for _ in range(n_bootstrap):
    crps_scores = []
    
    for i in range(n_seeds):
        # Resample indices with replacement
        n_samples = y_val[i]['max'].shape[0]
        resampled_indices = resample(range(n_samples), replace=True, n_samples=n_samples)
        
        # Resampled validation data and predicted mean
        y_val_resampled = y_val[i]['max'][resampled_indices]
        mean_pred_resampled = np.array([y_train[i]['max'].mean(axis=0) for _ in resampled_indices])
        
        # Evaluate CRPS score for the resampled data
        crps = np.mean(crps_ensemble(y_val_resampled, mean_pred_resampled))
        crps_scores.append(crps)
    
    # Average CRPS across all seeds for this bootstrap iteration
    bootstrap_crps.append(np.mean(crps_scores))

# Final CRPS statistics
mean_crps = np.mean(bootstrap_crps)
std_crps = np.std(bootstrap_crps)

In [8]:
mean_crps

4.706198990643481

In [9]:
resampled_indices

[5, 5, 6, 1, 6, 8, 2, 0, 2]

## RF

### 20

#### Deterministic

In [21]:
# Initialize variables
selected_pcs = []  # List to store selected PCs
remaining_vars = list(pcs_train[0].keys())  # All variables initially available for selection
best_val_rmse = float('inf')  # Start with a very high RMSE value

while remaining_vars:
    best_var = None
    best_pc_index = None
    best_mean_val_rmse = best_val_rmse

    # Test each variable
    for var in tqdm(remaining_vars):
        nPC = pcs_train[0][var].shape[1]  # Number of PCs for this variable

        # Test each PC of the variable
        for pc_index in range(nPC):
            mean_score = []

            # Evaluate using all seeds
            for iseed in range(7):
                # Prepare data for the current candidate PC
                candidate_features = [pcs_train[iseed][sel_var][:, [pc_idx]] 
                                      for sel_var, pc_idx in selected_pcs]
                candidate_features.append(pcs_train[iseed][var][:, [pc_index]])
                X_train_subset = np.hstack(candidate_features)
                X_val_subset = np.hstack(
                    [pcs_val[iseed][sel_var][:, [pc_idx]] for sel_var, pc_idx in selected_pcs] +
                    [pcs_val[iseed][var][:, [pc_index]]]
                )

                # Train a model and evaluate
                model = RandomForestRegressor(random_state=seed)
                grid_search = GridSearchCV(
                    estimator=model,
                    param_grid=param_grid,
                    scoring="neg_root_mean_squared_error",  # Use RMSE as scoring metric
                    cv=3,  # Inner cross-validation
                )
                grid_search.fit(X_train_subset, y_train[iseed][target_cat])
                
                # Use the best model to predict
                model = grid_search.best_estimator_
                #model.fit(X_train_subset, y_train[iseed][target_cat])
                y_pred = model.predict(X_val_subset)

                # Calculate the validation RMSE
                val_rmse = mean_squared_error(y_val[iseed][target_cat], y_pred, squared=False)
                mean_score.append(val_rmse)

            # Compute the mean validation RMSE across seeds
            mean_val_rmse = np.mean(mean_score)

            # Update the best PC if this one performs better
            if mean_val_rmse < best_mean_val_rmse:
                best_mean_val_rmse = mean_val_rmse
                best_var = var
                best_pc_index = pc_index

    # Check if we found a PC that improves validation RMSE
    if best_var and best_mean_val_rmse < best_val_rmse:
        # Add the best-performing PC to the selected set
        selected_pcs.append((best_var, best_pc_index))
        remaining_vars.remove(best_var)
        best_val_rmse = best_mean_val_rmse
        print(f"Selected PC: {best_var}_PC{best_pc_index + 1}, Mean Val RMSE: {best_mean_val_rmse}")
    else:
        print("No improvement. Stopping feature selection.")
        break


100%|██████████| 20/20 [10:45<00:00, 32.27s/it]


Selected PC: geopotential_500_max_PC10, Mean Val RMSE: 6.443124888320203


100%|██████████| 19/19 [10:30<00:00, 33.18s/it]


Selected PC: 2m_dewpoint_temperature_std_PC2, Mean Val RMSE: 5.921249557472562


100%|██████████| 18/18 [10:12<00:00, 34.00s/it]


...

Selected PC: mean_sea_level_pressure_mean_PC6, Mean Val RMSE: 5.569124719086719


100%|██████████| 10/10 [06:27<00:00, 38.71s/it]


Selected PC: surface_latent_heat_flux_mean_PC6, Mean Val RMSE: 5.49754599271219


100%|██████████| 9/9 [05:52<00:00, 39.21s/it]

No improvement. Stopping feature selection.


#### probabilistic

In [30]:
from properscoring import crps_ensemble  # For CRPS calculation
import numpy as np

# Initialize variables
selected_pcs = []  # List to store selected PCs as (variable, pc_index) tuples
remaining_vars = list(pcs_train[0].keys())  # Variables initially available for selection
best_val_crps = float('inf')  # Start with a very high CRPS value

while remaining_vars:
    best_var = None
    best_pc_index = None
    best_mean_val_crps = best_val_crps

    # Test each variable
    for var in remaining_vars:
        nPC = pcs_train[0][var].shape[1]  # Number of PCs for this variable

        # Test each PC of the variable
        for pc_index in range(nPC):
            mean_score = []

            # Evaluate using all seeds
            for iseed in range(7):
                # Prepare data for the current candidate PC
                candidate_features = [pcs_train[iseed][sel_var][:, [pc_idx]] 
                                      for sel_var, pc_idx in selected_pcs]
                candidate_features.append(pcs_train[iseed][var][:, [pc_index]])
                X_train_subset = np.hstack(candidate_features)
                X_val_subset = np.hstack(
                    [pcs_val[iseed][sel_var][:, [pc_idx]] for sel_var, pc_idx in selected_pcs] +
                    [pcs_val[iseed][var][:, [pc_index]]]
                )

                # Train a probabilistic model
                model = RandomForestRegressor(n_estimators=100, random_state=seed)
                model.fit(X_train_subset, y_train[iseed][target_cat])
                # Use the best model to predict
                y_pred_ensemble = np.array([tree.predict(X_val_subset) for tree in model.estimators_])

                # Calculate the CRPS score
                crps = np.mean([crps_ensemble(y_val[iseed][target_cat][i], y_pred_ensemble[:, i, :].T) 
                                for i in range(len(y_val[iseed][target_cat]))])
                mean_score.append(crps)

            # Compute the mean CRPS across seeds
            mean_val_crps = np.mean(mean_score)

            # Update the best PC if this one performs better
            if mean_val_crps < best_mean_val_crps:
                best_mean_val_crps = mean_val_crps
                best_var = var
                best_pc_index = pc_index

    # Check if we found a PC that improves validation CRPS
    if best_var and best_mean_val_crps < best_val_crps:
        # Add the best-performing PC to the selected set
        selected_pcs.append((best_var, best_pc_index))
        remaining_vars.remove(best_var)  # Remove the variable from remaining_vars
        best_val_crps = best_mean_val_crps
        print(f"Selected PC: {best_var}_PC{best_pc_index + 1}, Mean Val CRPS: {best_mean_val_crps}")
    else:
        print("No improvement. Stopping feature selection.")
        break


Selected PC: geopotential_500_max_PC10, Mean Val CRPS: 4.404707118874361
Selected PC: geopotential_500_mean_PC9, Mean Val CRPS: 3.729450611336759
Selected PC: surface_pressure_mean_PC11, Mean Val CRPS: 3.5516674759429776
Selected PC: geopotential_1000_std_PC10, Mean Val CRPS: 3.4916252511325006
Selected PC: 2m_dewpoint_temperature_std_PC10, Mean Val CRPS: 3.430094883373766
Selected PC: surface_latent_heat_flux_min_PC4, Mean Val CRPS: 3.394558978769711
Selected PC: 10m_u_component_of_wind_max_PC2, Mean Val CRPS: 3.3793392212730993
Selected PC: mean_sea_level_pressure_min_PC9, Mean Val CRPS: 3.338790111026809
Selected PC: surface_pressure_max_PC8, Mean Val CRPS: 3.3204019809062046
Selected PC: mean_sea_level_pressure_std_PC10, Mean Val CRPS: 3.2840720501433087
Selected PC: 10m_v_component_of_wind_max_PC2, Mean Val CRPS: 3.28273212527452
Selected PC: geopotential_1000_min_PC5, Mean Val CRPS: 3.2644896778478953
Selected PC: mean_surface_latent_heat_flux_min_PC4, Mean Val CRPS: 3.2580280566